In [1]:
import datetime as dt
import src.analytics2 as a
import src.visualize2 as v
import src.definitions as d

# Context
date_from = dt.date(2022, 12, 9)
date_to = dt.date(2022, 12, 31)
lobby = d.Lobby.XMATCH

f"{str(date_from)} <= date < {str(date_to)}"


'2022-12-09 <= date < 2022-12-31'

In [2]:
details = a.read_details_from_to(date_from, date_to, lobby)
details = details[details["game-ver"] == "2.0.1"]
details = a.add_orchestration_columns(details)
details["mode"].value_counts()


area      10054
yagura     6893
hoko       6604
asari      4413
Name: mode, dtype: int64

In [64]:
df = details[
    [
        "x-power",
        "A1-weapon",
        "A2-weapon",
        "A3-weapon",
        "A4-weapon",
        "B1-weapon",
        "B2-weapon",
        "B3-weapon",
        "B4-weapon",
        "A-orch",
        "B-orch",
        "weapon-match",
        "pool-match",
        "weapon-type-match",
        "weapon-range-match",
        "orch-diff",
    ]
].copy()


In [65]:
df["pool-match"].value_counts(normalize=True).mul(100).round(1)


4    90.9
3     6.8
2     1.9
1     0.3
0     0.1
Name: pool-match, dtype: float64

In [66]:
df["weapon-type-match"].value_counts(normalize=True).mul(100).round(1)


4    99.5
2     0.5
Name: weapon-type-match, dtype: float64

In [67]:
df["weapon-range-match"].value_counts(normalize=True).mul(100).round(1)


4    92.4
3     7.1
2     0.4
1     0.0
Name: weapon-range-match, dtype: float64

In [68]:
total = len(df.index)
total

27964

In [69]:
cnt = df.groupby(["pool-match", "weapon-type-match", "weapon-range-match"]).count()
cnt = cnt["A1-weapon"].to_frame().set_axis(["count"], axis=1)
cnt = cnt.div(total).mul(100).reset_index().sort_values("count", ascending=False)
cnt

,pool-match,weapon-type-match,weapon-range-match,count
11,4,4,4,90.916893
10,3,4,3,6.830210
9,2,4,4,1.040624
8,2,4,2,0.436275
7,2,2,4,0.368331
4,1,4,3,0.236018
0,0,4,4,0.085825
6,2,2,3,0.039336
1,1,2,3,0.025032
5,2,2,2,0.010728


In [108]:
high = df[df["x-power"] > 0]
df2 = high[(high["pool-match"] == 2) & (high["weapon-type-match"] == 2) & (high["weapon-range-match"] == 4)].sort_values(["weapon-match", "x-power"], ascending=False)
df2

,x-power,A1-weapon,A2-weapon,A3-weapon,A4-weapon,B1-weapon,B2-weapon,B3-weapon,B4-weapon,A-orch,B-orch,weapon-match,pool-match,weapon-type-match,weapon-range-match,orch-diff
5833,2530.0,screwslosher,variableroller,liter4k,jimuwiper,liter4k,screwslosher,splatspinner_collabo,prime_collabo,CRRb,CSSb,2,2,2,4,"([R, R], [S, S])"
11097,2420.0,sharp,bottlegeyser,lact450,prime_collabo,campingshelter,sharp,variableroller,bottlegeyser,SSSs,RRSs,2,2,2,4,"([S, S], [R, R])"
26585,2400.0,dualsweeper,kelvin525,longblaster,nautilus47,dynamo,longblaster,jimuwiper,dualsweeper,BSSS,BRRS,2,2,2,4,"([S, S], [R, R])"
6347,2240.0,variableroller,prime_collabo,splatspinner_collabo,kelvin525,variableroller,variableroller,jimuwiper,kelvin525,RSSS,RRRS,2,2,2,4,"([S, S], [R, R])"
7927,2200.0,sharp,splatspinner,dualsweeper,prime_collabo,prime_collabo,variableroller,sharp,jimuwiper,SSSs,RRSs,2,2,2,4,"([S, S], [R, R])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24230,1690.0,52gal,furo,nautilus47,liter4k,jimuwiper,splatroller,liter4k_scope,longblaster,BCSs,BCRr,0,2,2,4,"([S, s], [R, r])"
12446,1640.0,sputtery_hue,wakaba,prime_collabo,splatspinner_collabo,bold,jimuwiper,variableroller,52gal,SSss,RRss,0,2,2,4,"([S, S], [R, R])"
5892,1630.0,jimuwiper,carbon,sshooter_collabo,variableroller,momiji,lact450,dualsweeper,splatroller,RRrs,SSrs,0,2,2,4,"([R, R], [S, S])"
20154,1620.0,jimuwiper,wakaba,variableroller,96gal,bottlegeyser,promodeler_rg,lact450,prime_collabo,RRSs,SSSs,0,2,2,4,"([R, R], [S, S])"
